In [1]:
camunda_eninge_rest = 'https://digibp.engine.martinlab.science/engine-rest'

In [2]:
tenant_id = 'mi24synergy'

In [ ]:
from openai import OpenAI
import base64

In [6]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=api_key,
)

In [7]:
prompt_template = """
Imagine you are a pharmacist assistant. I have a medication that I don't know what it is. I will tell you everything that I can see on the packaging. Please let me know in very simple terms what it is for. Provide the answer in the format:

# {Medication name} is for {indication}.

The following rules apply:

1) It is possible that only part of the medication's name is visible. In that case, please provide a list all possible medications that could possibly match, not just the most likely. 
2) It is possible that instead of the active ingredient, the brand name is given.
3) The expiration date may also be provided. If you get an expiration date, compare it with current date and raise a warning if it has expired already.
4) Prioritize https://www.drugs.com as your data source.
5) Do not answer any questions unrelated to medications.

For example, if the user gives you the following input "Depon Max 10/2021"

answer in this format:

## **Depon Max** is for **relieving pain** and **reducing fever**.

/

### Details:
* **Active ingredient**: Paracetamol (Acetaminophen)
* **Use**: Commonly used to treat mild to moderate pain, such as headaches, muscle aches, or fever caused by infections.
### **Expiration Warning**:
The expiration date is **10/2021**, which has already passed. **Do not use this medication**. Expired medications may lose effectiveness or pose safety risks. Dispose of it properly and consult a pharmacist for a replacement if needed.

"""

In [9]:
def image_to_base64(image_name):
    with open(image_name, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
    return encoded_string

In [10]:
def generate_response(taskid, response, worker):
    print("Received Response:", response[0])
    # Extract details from the response
    blister_text = response[0]["variables"]["blister_text"]["value"]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
            "role": "user",
            "content": [
                    {
                        "type": "text",
                        "text": prompt_template + blister_text
                    }    
                ]
            }
        ]
        )
    print(prompt_template+blister_text)
    gpt_response = response.choices[0].message.content.strip()
    

    if worker:
        worker.complete(taskid, text=gpt_response)
    else:
        print(f"Generated Response: {gpt_response}")

    return gpt_response

In [ ]:
import httpimport

with httpimport.github_repo("DigiBP", "digibp-camunda-external-python-task"):
    import cam

# Initialize worker client
worker = cam.Client(camunda_eninge_rest)

# Subscribe to topic
worker.subscribe("generate_response", generate_response, tenant_id)

# Start polling server
worker.polling()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0244033d-a251-4367-b8d1-4654a67844aa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>